In [1]:
import gdspy
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
gdsii = gdspy.GdsLibrary(infile='models/lineartrap-Laser_dicing.gds')

In [3]:
# gdspy.LayoutViewer(gdsii)

In [4]:
import triangle
import trimesh
from stl import mesh
cell = gdsii.cells['chip1']
nof = 0

faces=[]

for poly in cell.polygons:
    if poly.layers[0]==702:
        tri=triangle.triangulate({"vertices":poly.polygons[0]}, 'q30a')
        vectors = np.take(tri['vertices'], tri['triangles'], axis=0)
        faces.append(vectors)
        nof+=len(tri['vertices'])

faces = np.concatenate(faces, axis=0)
faces = np.insert(faces, 2, 0, 2)

In [5]:
# layer_mesh_data = np.zeros(len(faces), dtype=mesh.Mesh.dtype)
# layer_mesh_data['vectors'] = faces
# m = mesh.Mesh(layer_mesh_data)
# m.save('models/bowtie.stl')

In [6]:

mesh = trimesh.load_mesh("models/lineartrap-Laser_dicing.gds_chip1.stl")
nov, nof, nop = len(mesh.vertices), len(mesh.faces), len( trimesh.graph.connected_components(mesh.edges))
# faces and triagles are used interchangably

print("number of vertices: " + str(nov))
print("number of faces: " + str(nof))
print("number of connected parts: " + str(nop))
import display
%matplotlib qt

indices = list(range(99, 135))
indices.extend([91, 92, 93, 94, 83, 95, 96, 140 ,141, 135, 139])

display.part(mesh, indices)


number of vertices: 980
number of faces: 620
number of connected parts: 180


In [18]:
def extract_connected_parts(mesh, indices):
    """
    Extracts connected parts of a trimesh object and returns them as a new trimesh object.

    :param mesh: A trimesh.Trimesh object representing the original mesh.
    :param indices: A list of indices for the connected parts to be included in the new mesh.
    :return: A new trimesh.Trimesh object that is a submesh of the connected parts.
    """
    # Split the mesh into disconnected components
    connected_components = mesh.split(only_watertight=False)

    # Verify indices are within the valid range
    if not all(0 <= idx < len(connected_components) for idx in indices):
        raise ValueError("One or more selected indices are out of range.")
    
    # Select the desired components
    selected_components = trimesh.util.concatenate(
        [connected_components[i] for i in indices]
    )

    return selected_components


new_mesh=extract_connected_parts(mesh, indices)

new_mesh.vertices/=1000
# change unit from [um] to [mm]

new_mesh.export("models/bowtie.stl")

new_mesh=trimesh.load_mesh("models/bowtie.stl")

display.part(new_mesh)


In [13]:
import numpy as np
import trimesh
import display
mesh5 = trimesh.load("models/bowtie_ie5.stl")
# display.part(mesh5, ground)

In [16]:
upper = [43, 42, 41, 40, 39, 38, 7, 11, 6, 37, 36, 35, 34, 5, 10, 4, 33, 32, 31, 30, 29, 44]
lower = [45, 28, 27, 26, 25, 24, 3, 9, 2, 23, 22, 21, 20, 1, 8, 0, 19, 18, 17, 16, 15, 46]
len(upper), len(lower)

rf=[12]
ground=[14, 13]

pairs = {}

for i, (u, l) in enumerate(zip(upper, lower)):
    pairs[i]=(u, l)

pairs["rf"]=rf
pairs["ground"]=ground

idx2sector=np.zeros(47)
idx2sector[upper]=2
idx2sector[lower]=3
idx2sector[rf]=1
idx2sector[ground]=0

electrodes = trimesh.graph.split(mesh5,False)
xcenter=np.array([(electrode.vertices[:,0].max()+electrode.vertices[:, 0].min())/2 for electrode in electrodes])


np.savez("models\\bowtie_ie5", pairs=pairs, idx2sector=idx2sector, dc_sectors=[2, 3], xcenter=xcenter)
np.savez("models\\bowtie_ie2", pairs=pairs, idx2sector=idx2sector, dc_sectors=[2, 3], xcenter=xcenter)
np.savez("models\\bowtie_ie3", pairs=pairs, idx2sector=idx2sector, dc_sectors=[2, 3], xcenter=xcenter)
np.savez("models\\bowtie_ie4", pairs=pairs, idx2sector=idx2sector, dc_sectors=[2, 3], xcenter=xcenter)

In [17]:
electrodes[32].vertices

# xcenter[32]

TrackedArray([[0.55601001, 0.56898999, 0.        ],
              [0.55601001, 0.49811873, 0.        ],
              [0.57856357, 0.52369481, 0.        ],
              [0.67598999, 0.19101   , 0.        ],
              [0.67598999, 0.26188123, 0.        ],
              [0.65343648, 0.23630515, 0.        ],
              [0.55601001, 0.42724749, 0.        ],
              [0.55601001, 0.35637623, 0.        ],
              [0.57934272, 0.42289606, 0.        ],
              [0.62960649, 0.44877234, 0.        ],
              [0.5862208 , 0.47420746, 0.        ],
              [0.55601001, 0.2382575 , 0.        ],
              [0.55601001, 0.19101   , 0.        ],
              [0.60548639, 0.24293303, 0.        ],
              [0.64577919, 0.28579253, 0.        ],
              [0.67598999, 0.3327525 , 0.        ],
              [0.67598999, 0.40362376, 0.        ],
              [0.65265739, 0.33710399, 0.        ],
              [0.60239351, 0.31122768, 0.        ],
            